In [6]:
import pandas as pd
import numpy as np
from numpy.linalg import svd


df = pd.read_csv('ratings_small.csv')
df.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


Precisamos passar todos os dados para uma matriz

Exemplo 3 usuarios e 4 filmes

usuario1 = [nota_1_1, nota_1_2, nota_1_3, nota_1_4]

usuario2 = [nota_2_1, nota_2_2, nota_2_3, nota_2_4]

usuario3 = [nota_3_1, nota_3_2, nota_3_3, nota_3_4]


portanto nossa matiz A ficaria assim:

$$
A = \begin{bmatrix}
    nota_{1,1} & nota_{1,2} & nota_{1,3} & nota_{1,4} \\
    nota_{2,1} & nota_{2,2} & nota_{2,3} & nota_{2,4} \\
    nota_{3,1} & nota_{3,2} & nota_{3,3} & nota_{3,4} \\
\end{bmatrix}



In [25]:
'''
Exemplo 3 usuarios e 4 filmes

usuario1 = [nota_1_1, nota_1_2, nota_1_3, nota_1_4]
usuario2 = [nota_2_1, nota_2_2, nota_2_3, nota_2_4]
usuario3 = [nota_3_1, nota_3_2, nota_3_3, nota_3_4]

matriz A = [ [nota_1_1, nota_1_2, nota_1_3, nota_1_4],
                         [nota_2_1, nota_2_2, nota_2_3, nota_2_4],
                         [nota_3_1, nota_3_2, nota_3_3, nota_3_4]]
'''

# Criando a matriz de notas
matriz_notas = df.pivot_table(index='userId', columns='movieId', values='rating')
matriz_notas = matriz_notas.fillna(0)
matriz_notas.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.decomposition import TruncatedSVD

def svd_input(A, n_components):
    # Calculando a matriz de notas médias
    notas_medias = np.nanmean(A, axis=1)
    A_demeaned = A - notas_medias.reshape(-1, 1)

    # Calculando a SVD truncada
    svd = TruncatedSVD(n_components=n_components)
    svd.fit(A_demeaned)
    U = svd.transform(A_demeaned)
    sigma = svd.singular_values_
    Vt = svd.components_

    return U, sigma, Vt, notas_medias


In [39]:
U, sigma, Vt, notas_medias = svd_input(matriz_notas.values, 500) # quanto maior o n_components, mais precisa a recomendação

A_reconstruida = U @ np.diag(sigma) @ Vt + notas_medias.reshape(-1, 1)

# Calculando o erro
erro = np.sum((matriz_notas.values - A_reconstruida)**2)
print(f'Erro: {erro}')

# Primeiro componente principal (vetor coluna de U)
cp1 = U[:, 0]

# Segundo componente principal (vetor coluna de Vt)
cp2 = Vt.T[:, 1]

# Variância explicada pelos dois primeiros componentes principais
var_exp = (sigma[0]**2 + sigma[1]**2) / np.sum(sigma**2)
print(f'Variância explicada pelos dois primeiros CPs: {var_exp:.2%}')

Erro: 55857678379.88517
Variância explicada pelos dois primeiros CPs: 21.23%
